<a href="https://colab.research.google.com/github/Siddhi-S-Thakur/Movie-Review-Sentiment-Analysis/blob/main/movieReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("IMDB Dataset.csv", engine = 'python', on_bad_lines = "skip")

## Data preprocessing

In [ ]:
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

/tmp/ipython-input-1137712857.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
# Converting reviews to lowercase
df['review'] = df['review'].str.lower()

In [ ]:

# Removing HTML tags from reviews
import re

def remove_html(text):
    pattern = r'<.*?>'
    text = re.sub(pattern, '', text)
    return text

df['review'] = df['review'].apply(remove_html)


In [ ]:
df.head(2)

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1


In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_urls(text):
    pattern = r'https?://\S+|www\.\S+'
    return re.sub(pattern, '', text)

df['review'] = df['review'].astype(str).apply(remove_urls)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
X = df['review']
y = df['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### TF-IDF

In [ ]:
#transform the text data into numerical features using TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

### ML Models

In [ ]:
#Logistic Regression Model
log_model = LogisticRegression(max_iter=500)
log_model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=500)

In [ ]:
y_pred = log_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8802


In [ ]:
# Random Forest Model

from sklearn.ensemble import RandomForestClassifier

rnd_model = RandomForestClassifier(n_estimators=100, random_state=42)
rnd_model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = rnd_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8421


In [ ]:
#Decision Tree Model
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_tfidf, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
y_pred = dt_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7129


In [ ]:
# XGBoost Model
from xgboost import XGBClassifier

xgb_model = XGBClassifier(use_label_encoder= False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_tfidf, y_train)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:200: UserWarning: [15:53:55] WARNING: /__w/xgboost/xgboost/src/learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred = xgb_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8480


In [ ]:
#KNN Model
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_tfidf, y_train)


KNeighborsClassifier()

In [ ]:
y_pred = knn_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7353


In [ ]:
#SVM Model
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_tfidf, y_train)

SVC(kernel='linear', random_state=42)

In [ ]:
y_pred = svm_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8762


### DL tf-Keras Model

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Tokenize text data

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen = 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen = 200)

In [ ]:
# prepareing target variables

Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [ ]:
# Building LSTM model

model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim=128, input_shape=(200,)))
model.add(LSTM(128, dropout=0.2, recurrent_dropout = 0.2))
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
#Compiling the model

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#Training the model

model.fit(X_train, Y_train, epochs = 8, batch_size= 100, validation_split = 0.2)

Epoch 1/8
202/202 ━━━━━━━━━━━━━━━━━━━━ 152s 753ms/step - accuracy: 0.8804 - loss: 0.2971 - val_accuracy: 0.8479 - val_loss: 0.3528
Epoch 2/8
202/202 ━━━━━━━━━━━━━━━━━━━━ 152s 751ms/step - accuracy: 0.9063 - loss: 0.2458 - val_accuracy: 0.8733 - val_loss: 0.3158
Epoch 3/8
202/202 ━━━━━━━━━━━━━━━━━━━━ 152s 752ms/step - accuracy: 0.9254 - loss: 0.2047 - val_accuracy: 0.8677 - val_loss: 0.3240
Epoch 4/8
202/202 ━━━━━━━━━━━━━━━━━━━━ 151s 749ms/step - accuracy: 0.9364 - loss: 0.1793 - val_accuracy: 0.8681 - val_loss: 0.3358
Epoch 5/8
202/202 ━━━━━━━━━━━━━━━━━━━━ 152s 753ms/step - accuracy: 0.9359 - loss: 0.1728 - val_accuracy: 0.8618 - val_loss: 0.3580
Epoch 6/8
202/202 ━━━━━━━━━━━━━━━━━━━━ 151s 748ms/step - accuracy: 0.9567 - loss: 0.1294 - val_accuracy: 0.8614 - val_loss: 0.3689
Epoch 7/8
202/202 ━━━━━━━━━━━━━━━━━━━━ 203s 752ms/step - accuracy: 0.9650 - loss: 0.1060 - val_accuracy: 0.8588 - val_loss: 0.4259
Epoch 8/8
202/202 ━━━━━━━━━━━━━━━━━━━━ 151s 747ms/step - accuracy: 0.9678 - loss: 0